In [2]:
%matplotlib inline
import pandas as pd
import os
import os.path
import itertools
import matplotlib.pyplot as plt
import math
import pprint
import pprint as pp
import numpy as np

## 1.  LOAD DATA

In [24]:
DATA_PATH = os.path.abspath(os.path.join(os.pardir, "data/gapminder"))
DATA_PATH2 = os.path.abspath(os.path.join(os.pardir, "data"))
Child_death_FILE = "child-death.csv"
GINI_FILE = "gini-complete.csv"
#Year_in_school = "Years in school men 25-34.csv"
FERTILITY_FILE = "teen-fertility.csv"
COUNTRIES_FILE = "3-countries.csv"
fertility_df = pd.read_csv(os.path.join(DATA_PATH, FERTILITY_FILE), sep=";")
gini_df = pd.read_csv(os.path.join(DATA_PATH, GINI_FILE),sep=",")
child_death_df = pd.read_csv(os.path.join(DATA_PATH, Child_death_FILE), sep=";")
countries_df = pd.read_csv(os.path.join(DATA_PATH2, COUNTRIES_FILE), sep=";", usecols = ['Country (en)', "Continent"])

In [26]:
countries_df = countries_df.rename(columns ={"Country (en)" :"countries"})
countries_df.head(3)

,countries,Continent
0,Afghanistan,Asia
1,Egypt,Africa
2,Åland Islands,Europe


In [15]:
fertility_df.head(3)

,countries,Teen-fertility
0,Abkhazia,NaN
1,Afghanistan,115.0
2,Akrotiri and Dhekelia,NaN


In [16]:
gini_df.head(3)

,countries,Gini
0,Afghanistan,27.8
1,Lesotho,63.2
2,South_Africa,62.5


In [17]:
child_death_df.head(3)

,countries,child-death
0,Afghanistan,195.188070
1,Albania,10.526316
2,Algeria,18.090586


In [27]:
df_intermediate = pd.merge(gini_df, child_death_df,  how='inner', on=['countries'])
df_intermediate.head(3)


,countries,Gini,child-death
0,Afghanistan,27.8,195.188070
1,Lesotho,63.2,39.732421
2,Haiti,60.8,45.507790


In [28]:
df_final = pd.merge(df_intermediate, countries_df,  how='left', on=['countries'])
df_final.head(3)

,countries,Gini,child-death,Continent
0,Afghanistan,27.8,195.188070,Asia
1,Lesotho,63.2,39.732421,Africa
2,Haiti,60.8,45.507790,North America


In [30]:
df_final_cols = df_final.columns.tolist()
df_final_cols = df_final_cols[-1:] + df_final_cols[:-1]
df_final = df_final[df_final_cols]
df_final.head(3)


,Continent,countries,Gini,child-death
0,Asia,Afghanistan,27.8,195.188070
1,Africa,Lesotho,63.2,39.732421
2,North America,Haiti,60.8,45.507790


In [34]:
import json
from json import dumps

json_dict = {}
json_dict["name"] = "Json_file"
json_dict['children'] = []
for grp, grp_data in df_final.groupby('Continent'):
    #print ('grp_data',grp_data)
    grp_dict = {}
    #print (grp)
    grp_dict['name'] = grp
    grp_dict['children'] = []
    for cat, cat_data in grp_data.groupby('countries'):
        #print ("cat",cat)
        #print ("cat_data",cat_data)
        
        cat_dict = {}
         
        cat_dict['name'] = cat
        
        cat_dict['children'] = []
        #cat_dict['name'] = cat
        for p, p_data in cat_data.groupby('Gini'):
            cat_dict['gini'] =p//10
            p_data = p_data.drop(['Continent', 'countries'], axis=1).set_index('Gini')
            
            for d in p_data.to_dict(orient='records'):
                for key, value in d.items():
                    d_prime ={}
                    d_prime ["name"] =key
                    d_prime ["size"] =value
                    cat_dict['children'].append(d_prime)
        grp_dict['children'].append(cat_dict)
    json_dict['children'].append(grp_dict)
json_out = dumps(json_dict)
parsed = json.loads(json_out)    

In [35]:
json_out

'{"name": "Json_file", "children": [{"name": "Africa", "children": [{"name": "Algeria", "children": [{"name": "child-death", "size": 18.0905859}], "gini": 3.0}, {"name": "Angola", "children": [{"name": "child-death", "size": 166.79505840000002}], "gini": 4.0}, {"name": "Benin", "children": [{"name": "child-death", "size": 82.27262885}], "gini": 3.0}, {"name": "Botswana", "children": [{"name": "child-death", "size": 14.95224178}], "gini": 6.0}, {"name": "Burundi", "children": [{"name": "child-death", "size": 119.3588838}], "gini": 4.0}, {"name": "Cameroon", "children": [{"name": "child-death", "size": 95.87426717}], "gini": 4.0}, {"name": "Chad", "children": [{"name": "child-death", "size": 153.56249340000002}], "gini": 4.0}, {"name": "Djibouti", "children": [{"name": "child-death", "size": 57.06724010000001}], "gini": 4.0}, {"name": "Egypt", "children": [{"name": "child-death", "size": 8.749506845}], "gini": 3.0}, {"name": "Ethiopia", "children": [{"name": "child-death", "size": 64.608

In [37]:
import io

# process Unicode text
with io.open('gini_child_death_.json','w',encoding='utf8') as f:
    json.dump(parsed, f)

In [ ]:
##FIN

## 2. PREPARE DATA

### 2.1 Data selection

In [5]:
gini_df = gini_df.rename(columns={"GINI index": "country"})
Year_in_school_df = Year_in_school_df.rename(columns ={"Row Labels" :"country"})
countries_df = countries_df.rename(columns ={"Country (en)" :"country"})
murder_df = murder_df.rename(columns={"Murder_per_100000_age_adjusted": "country"})
murder_df.head(3)

,country,1950,1951,1952,1953,1954,1955,1956,1957,1958,...,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005
0,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"4,650731",NaN,"3,83731842","3,83731842"
1,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"5,972975","8,115988691","6,681117058","7,326701331"
2,Algeria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"13,10029","14,41026872","9,806334496","4,389205469"


### 2.2 Load countries data

In [6]:
# vérification du nombre de pays dans chacune des dataframes
print ("Nombre de pays dans countries_df est {}".format (len(list (countries_df["country"].unique()))))
print ("Nombre de pays dans gini_df est {}".format (len(list (gini_df["country"].unique()))))
print ("Nombre de pays dans Year_in_school_df est {}".format (len(list (Year_in_school_df["country"].unique()))))
print ("Nombre de pays dans murder_df est {}".format (len(list (murder_df["country"].unique()))))

Nombre de pays dans countries_df est 248
Nombre de pays dans gini_df est 227
Nombre de pays dans Year_in_school_df est 175
Nombre de pays dans murder_df est 206


In [7]:
# Récupère le continent auquel appartient chaque pays


###### GINI INDEX 
gini_df_complet =pd.merge(gini_df, countries_df, on='country', how='outer')
gini_df_complet_cols = gini_df_complet.columns.tolist()


## Réorganisation des colonnes :mettre la colonne Continent en premier.
gini_df_complet_cols = gini_df_complet_cols[-1:] + gini_df_complet_cols[:-1]
gini_df_complet = gini_df_complet[gini_df_complet_cols]

###### Year in school 
Year_in_school_df_complet = pd.merge(Year_in_school_df, countries_df, on='country', how='outer')
Year_in_school_df_complet_cols = Year_in_school_df_complet.columns.tolist()

## Réorganisation des colonnes :mettre la colonne Continent en premier.
Year_in_school_df_complet_cols = Year_in_school_df_complet_cols[-1:] + Year_in_school_df_complet_cols[:-1]
Year_in_school_df_complet = Year_in_school_df_complet[Year_in_school_df_complet_cols]

###### Murder
Murder_df_complet = pd.merge(murder_df, countries_df, on='country', how='outer')
Murder_df_complet_cols = Murder_df_complet.columns.tolist()

## Réorganisation des colonnes :mettre la colonne Continent en premier.
Murder_df_complet_cols = Murder_df_complet_cols[-1:] + Murder_df_complet_cols[:-1]
Murder_df_complet = Murder_df_complet[Murder_df_complet_cols]



Murder_df_complet.head(3)

,Continent,country,1950,1951,1952,1953,1954,1955,1956,1957,...,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005
0,Asia,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"4,650731",NaN,"3,83731842","3,83731842"
1,Europe,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"5,972975","8,115988691","6,681117058","7,326701331"
2,Africa,Algeria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"13,10029","14,41026872","9,806334496","4,389205469"


In [8]:
#gini_df_complet.transpose().head()
def commun_columns (dataframe_1 ,dataframe_2):
    print("Dimensions initiales ...")
    print("   Taille gini colonnes : ", str(len(dataframe_1.columns.values)))
    print("   Taille year school colonnes : ", str(len(dataframe_2.columns.values)))
    print()

    gini_not_in_year_school = [ colonne for colonne in dataframe_1.columns.values if colonne not in dataframe_2.columns.values ]

    year_school_not_in_gini = [ colonne  for colonne in dataframe_2.columns.values if colonne not in dataframe_1.columns.values ]

    print ("colonnes dans gini et pas dans mean year school {}".format(gini_not_in_year_school))
    print ("colonnes dans mean year school et pas dans gini {}".format(year_school_not_in_gini))

    print("Suppression des colonnes non communes ...")

    dataframe_1 = dataframe_1.drop(gini_not_in_year_school, axis = 1 )
    dataframe_2 = dataframe_2.drop(year_school_not_in_gini, axis = 1 )

    print("\nDimensions finales")
    print("    Taille gini colonnes : ", str(len(dataframe_1.columns.values)))
    print("    Taille year school colonnes : ", str(len(dataframe_2.columns.values)))
    return dataframe_1, dataframe_2

In [9]:
gini_df_complet ,Year_in_school_df_complet =commun_columns (gini_df_complet ,Year_in_school_df_complet)

Dimensions initiales ...
   Taille gini colonnes :  32
   Taille year school colonnes :  42

colonnes dans gini et pas dans mean year school []
colonnes dans mean year school et pas dans gini ['1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '2008', '2009']
Suppression des colonnes non communes ...

Dimensions finales
    Taille gini colonnes :  32
    Taille year school colonnes :  32


### Rajout de la table "Murder" au deux autres tables "mean year school" et "Gini Index" et garder que les colonnes en commun 

In [10]:
Murder_not_in_gini_and_year = [ colonne  for colonne in Murder_df_complet.columns.values if colonne not in gini_df_complet.columns.values ]
gini_and_year_not_Murder = [ colonne  for colonne in gini_df_complet.columns.values if colonne not in Murder_df_complet.columns.values ]
print ("colonnes dans Murder  pas dans mean year school et gini  {}".format(Murder_not_in_gini_and_year))
print ("colonnes dans mean year school et gini pas dans Murder {}".format(gini_and_year_not_Murder))

print("Suppression des colonnes non communes ...")

gini_df_complet = gini_df_complet.drop(gini_and_year_not_Murder, axis = 1 )
Year_in_school_df_complet = Year_in_school_df_complet.drop(gini_and_year_not_Murder, axis = 1 )

Murder_df_complet = Murder_df_complet.drop(Murder_not_in_gini_and_year ,axis =1)

print("\nDimensions finales")
print("    Taille gini colonnes : ", str(len(gini_df_complet.columns.values)))
print("    Taille year school colonnes : ", str(len(Year_in_school_df_complet.columns.values)))
print("    Taille Murder colonnes : ", str(len(Murder_df_complet.columns.values)))

colonnes dans Murder  pas dans mean year school et gini  ['1950', '1951', '1952', '1953', '1954', '1955', '1956', '1957', '1958', '1959', '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977']
colonnes dans mean year school et gini pas dans Murder ['2006', '2007']
Suppression des colonnes non communes ...

Dimensions finales
    Taille gini colonnes :  30
    Taille year school colonnes :  30
    Taille Murder colonnes :  30


## Transposition des Trois Dataframe 

In [11]:
def transposition(dataframe):
    dataframe = dataframe.transpose()
    dataframe.drop(dataframe.index[0],inplace =True)
    dataframe.columns = dataframe.iloc[0]
    dataframe.drop(dataframe.index[0], inplace=True)

    return dataframe

### Gini Index 

In [12]:
gini_df_complet = transposition(gini_df_complet)
gini_df_complet

country,Afghanistan,Albania,Algeria,American Samoa,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Aruba,...,Taiwan,Tokelau,Czech Republic,Turks and Caicos Islands,Tuvalu,Holy See (Vatican City),Wallis and Futuna,Christmas Island,Western Sahara,Central African Republic
1978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1981,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1982,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1986,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.51,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1987,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Year_in_school

In [17]:
Year_in_school_df_complet = transposition(Year_in_school_df_complet)

### Murder

In [18]:
Murder_df_complet = transposition(Murder_df_complet)

In [19]:
Murder_df_complet.head()

country,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Svalbard,Taiwan,Tokelau,Czech Republic,Turks and Caicos Islands,Holy See (Vatican City),Wallis and Futuna,Christmas Island,Western Sahara,Central African Republic
1978,NaN,NaN,NaN,NaN,NaN,NaN,"5,861383",NaN,"1,859457","1,514639",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1979,NaN,NaN,NaN,NaN,NaN,NaN,"3,725508",NaN,"1,875632","1,316937",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980,NaN,NaN,NaN,NaN,NaN,NaN,"3,678448",NaN,"1,919996","1,207952",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1981,NaN,NaN,NaN,NaN,NaN,NaN,"4,079281",NaN,"1,902391","1,25449",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1982,NaN,NaN,NaN,NaN,NaN,NaN,"4,290652",NaN,"1,997273","1,530068",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
gini_df_complet ,Year_in_school_df_complet = commun_columns(gini_df_complet,Year_in_school_df_complet)

Dimensions initiales ...
   Taille gini colonnes :  285
   Taille year school colonnes :  260

colonnes dans gini et pas dans mean year school ['Channel Islands', 'East Asia & Pacific', 'Euro area', 'Europe & Central Asia', 'Faeroe Islands', 'Heavily indebted poor countries (HIPC)', 'High income', 'High income: nonOECD', 'High income: OECD', 'Hong Kong, China', 'Korea, Dem. Rep.', 'Latin America & Caribbean', 'Least developed countries: UN classification', 'Low & middle income', 'Low income', 'Lower middle income', 'Macao, China', 'Micronesia, Fed. Sts.', 'Middle East & North Africa', 'Middle income', 'South Asia', 'Sub-Saharan Africa', 'Upper middle income', 'Virgin Islands (U.S.)', 'World']
colonnes dans mean year school et pas dans gini []
Suppression des colonnes non communes ...

Dimensions finales
    Taille gini colonnes :  260
    Taille year school colonnes :  260


In [21]:
Murder_not_in_gini_and_year = [ colonne  for colonne in Murder_df_complet.columns.values if colonne not in gini_df_complet.columns.values ]
gini_and_year_not_Murder = [ colonne  for colonne in gini_df_complet.columns.values if colonne not in Murder_df_complet.columns.values ]
print ("colonnes dans Murder  pas dans mean year school et gini  {}".format(Murder_not_in_gini_and_year))
print ("colonnes dans mean year school et gini pas dans Murder {}".format(gini_and_year_not_Murder))

print("Suppression des colonnes non communes ...")

gini_df_complet = gini_df_complet.drop(gini_and_year_not_Murder, axis = 1 )
Year_in_school_df_complet = Year_in_school_df_complet.drop(gini_and_year_not_Murder, axis = 1 )

Murder_df_complet = Murder_df_complet.drop(Murder_not_in_gini_and_year ,axis =1)

print("\nDimensions finales")
print("    Taille gini colonnes : ", str(len(gini_df_complet.columns.values)))
print("    Taille year school colonnes : ", str(len(Year_in_school_df_complet.columns.values)))
print("    Taille Murder colonnes : ", str(len(Murder_df_complet.columns.values)))

colonnes dans Murder  pas dans mean year school et gini  ['Cook Is', 'Czechoslovakia', 'East Germany', 'Hong Kong, China', 'Korea, Dem. Rep.', 'Micronesia, Fed. Sts.', 'Serbia and Montenegro', 'USSR', 'Virgin Islands (U.S.)', 'West Germany', 'Yugoslavia']
colonnes dans mean year school et gini pas dans Murder ['West Bank and Gaza']
Suppression des colonnes non communes ...

Dimensions finales
    Taille gini colonnes :  259
    Taille year school colonnes :  259
    Taille Murder colonnes :  259


In [22]:
gini_df_complet.head()

country,Afghanistan,Albania,Algeria,American Samoa,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Aruba,...,Taiwan,Tokelau,Czech Republic,Turks and Caicos Islands,Tuvalu,Holy See (Vatican City),Wallis and Futuna,Christmas Island,Western Sahara,Central African Republic
1978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1981,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1982,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
def stack_columns (dataframe ,label):
    dataframe = dataframe.stack()
    dataframe = dataframe.to_frame().reset_index()
    dataframe = dataframe.rename(columns= {0: label ,'level_0':'year'})
    return dataframe

#### stack gini 

In [24]:
gini_df_complet = stack_columns(gini_df_complet ,label ='gini')
gini_df_complet.head(3)

,year,country,gini
0,1979,Panama,48.74
1,1980,Madagascar,46.85
2,1981,Brazil,57.48


#### stack year in school 

In [25]:
Year_in_school_df_complet = stack_columns(Year_in_school_df_complet ,label ='mean_y_s')
Year_in_school_df_complet.head(3)

,year,country,mean_y_s
0,1978,Afghanistan,"1,5"
1,1978,Albania,"7,8"
2,1978,Algeria,"2,8"


#### Murder 

In [26]:
Murder_df_complet = stack_columns(Murder_df_complet ,label ='murder')
Murder_df_complet.head(3)

,year,country,murder
0,1978,Argentina,"5,861383"
1,1978,Australia,"1,859457"
2,1978,Austria,"1,514639"


In [27]:
def remove_comma(s):
    """
    Remplace les virgules d'une chaîne de caractères par des points.
    Utile pour les nombres décimaux écrits avec une virgule qui devront être castés de String vers Float
    """
    if not (type(s) is str or type(s) is np.str_):
        # just returns it untouched
        return s
    return s.replace(",", ".")

Year_in_school_df_complet.head()

,year,country,mean_y_s
0,1978,Afghanistan,"1,5"
1,1978,Albania,"7,8"
2,1978,Algeria,"2,8"
3,1978,Angola,"3,2"
4,1978,Antigua and Barbuda,"9,5"


#### remove gamma 

In [28]:
Murder_df_complet["murder"]= Murder_df_complet["murder"].apply(remove_comma).apply(float)
Year_in_school_df_complet["mean_y_s"] = Year_in_school_df_complet["mean_y_s"].apply(remove_comma).apply(float)
gini_df_complet["gini"] = gini_df_complet["gini"].apply(remove_comma).apply(float)
gini_df_complet.head()

,year,country,gini
0,1979,Panama,48.74
1,1980,Madagascar,46.85
2,1981,Brazil,57.48
3,1981,Costa Rica,47.49
4,1981,Thailand,45.22


#### Merging Three dataframe 

In [29]:
df_intermediate = pd.merge(Year_in_school_df_complet, gini_df_complet,  how='inner', on=['year','country'])
df_final = pd.merge(df_intermediate, Murder_df_complet,  how='inner', on=['year','country'])
df_final.head()

,year,country,mean_y_s,gini,murder
0,1981,Costa Rica,6.8,47.49,5.962232
1,1981,Venezuela,6.5,55.82,11.032220
2,1982,Brazil,4.8,58.15,13.837680
3,1983,Brazil,5.0,58.40,14.666470
4,1984,Brazil,5.1,58.43,16.074800


In [30]:
df_final = pd.merge(df_final, countries_df,  how='left', on=['country'])


### générer un fichier Json pour l'année 2005

In [31]:
df_final_filtred = df_final[df_final["year"]=="2005"]
df_final_filtred= df_final_filtred.drop('year', 1)

In [32]:
#df_final_filtred = df_final_filtred.set_index('Continent')

#grouped_df = df_final_filtred.groupby(df_final_filtred.index)
#grouped_df.head(3)

In [33]:
import json
from json import dumps

json_dict = {}
json_dict["name"] = "Json_file"
json_dict['children'] = []
for grp, grp_data in df_final_filtred.groupby('Continent'):
    #print ('grp_data',grp_data)
    grp_dict = {}
    #print (grp)
    grp_dict['name'] = grp
    grp_dict['children'] = []
    for cat, cat_data in grp_data.groupby('country'):
        #print ("cat",cat)
        #print ("cat_data",cat_data)
        
        cat_dict = {}
         
        cat_dict['name'] = cat
        
        cat_dict['children'] = []
        #cat_dict['name'] = cat
        for p, p_data in cat_data.groupby('gini'):
            cat_dict['gini'] =p//10
            p_data = p_data.drop(['Continent', 'country'], axis=1).set_index('gini')
            
            for d in p_data.to_dict(orient='records'):
                for key, value in d.items():
                    d_prime ={}
                    d_prime ["name"] =key
                    d_prime ["size"] =value
                    cat_dict['children'].append(d_prime)
        grp_dict['children'].append(cat_dict)
    json_dict['children'].append(grp_dict)
json_out = dumps(json_dict)
parsed = json.loads(json_out)        

In [34]:
type (json_out)

str

In [36]:
import io

# process Unicode text
with io.open('data_viz3.json','w',encoding='utf8') as f:
    json.dump(parsed, f)